### 필요한것들 다운 (모듈, 크롬옵션)

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver/usr/bin
!pip install webdriver_manager
!pip install chromedriver_autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,032 kB]
Hit:

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
from itertools import repeat
from itertools import groupby

import pandas as pd
import numpy as np

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # GUI 없는 환경에서는 headless 모드를 사용
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")

### [ 필독 ] 코드 설명 및 주의할 점

0. 수정해야 할 부분
- '######### 요 부분 수정' 에 각자 맡은 DJ 메인 페이지 링크
- for i in range(1,3) 부분에 몇 번째 플리부터 몇 개 가져올 건지
- pickle로 저장하는 부분 각자 경로에 맞춰서 수정

1. 코드 설명
  - 이번에는 굳이 함수화 시키지 않고 그냥 과정 그대로 남겨두기로 했습니다!
  - 코드에 '######### 요 부분 수정' 이라고 되어 있는 곳에 DJ 메인 페이지 링크 넣어주시면 됩니다
  - for i in range(1,3) -> 이 부분이 몇 개 플레이리스트 가져올지 명시하는 부분인데, 현재는 1번째, 2번째 플레이리스트 가져오도록 만들어둔 상태입니다. 이 부분의 숫자를 바꿔서 가져오면 됩니다. 다만, 시간이 꽤 걸리고 중간에 오류가 날 수도 있으니 조금씩 시도하는 걸 추천합니다..
  - 노래 30개 정도 있는 플레이리스트 하나당 7분 정도 걸립니다.

2. 주의할 점
  - 그리고 차트 크롤링이랑 마찬가지로 pickle로 저장하도록 만들어두었는데, 그 경로를 각자 맞게 수정하시기를 바랍니다
  - 앞서 말했듯 많은 작업이 한 번에 이루어지다 보니, 중간에 페이지가 제대로 로드되지 않아 오류가 날 수도 있습니다. 조금씩 돌리는 걸 추천.... 합니다.....
  - 혹시라도 중간에 오류가 날 상황을 대비하여 몇 번째 플레이리스트까지 저장했는지를 출력하도록 만들어두었습니다. 참고하여 다음 플레이리스트부터 이어 진행하시면 됩니다 !

### 플레이리스트별 노래 가져오기 크롤링

In [8]:
driver = webdriver.Chrome(options = chrome_options)

dj = 'boba' #####################################
# hashtags = []

for i in range(11,21):

  # 메인 페이지 불러오기
  driver.get('https://www.melon.com/mymusic/dj/mymusicdjplaylist_list.htm?memberKey=50700055') #####################################
  time.sleep(3)

  # playlist 고유 번호 가져오기 ----------------------------------
  link = driver.find_element(By.XPATH, f'//*[@id="pageList"]/div/div/table/tbody/tr[{i}]/td[4]/div/button') # i 번째 playlist 링크
  playlist = link.get_attribute('data-djcol-no') # playlist 고유 번호

  # playlist 안으로 이동 ----------------------------------
  driver.get(f'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq={playlist}')
  time.sleep(3)
  WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH,'//*[@id="frm"]/div/div/button[1]')))

  # 해시태그 가져오기
  hashtag_elements = driver.find_elements(By.CSS_SELECTOR, '.tag_list.type03 .tag_item')
  tags = [hashtag.text for hashtag in hashtag_elements]
  # hashtags.append(tags)

  # DJ 이름
  # dj = driver.find_element(By.XPATH, '//*[@id="conts"]/div[2]/div/div[2]/div[1]/div[2]/div/a').text

  # playlist 속 노래 개수
  songs = len(driver.find_elements(By.CLASS_NAME, 'rank')) # 총 노래 수 + 1

  titles = []
  artists = []
  lyrics = []
  whens = []
  likes = []

  # 각 노래마다 새로 창 열어가면서 진행 ----------------------------------
  for index in range(1,songs):

    driver.get(f'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq={playlist}')
    time.sleep(3)

    song_link = driver.find_element(By.XPATH, f'//*[@id="frm"]/div/table/tbody/tr[{index}]/td[7]/div/button')
    number = song_link.get_attribute('data-song-no') # 노래 고유 번호 가져오기

    driver.get(f'https://www.melon.com/song/detail.htm?songId={number}') # 상세 페이지 오픈
    time.sleep(3)

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'song_name')))

    title = driver.find_element(By.CLASS_NAME, 'song_name').text
    artist = driver.find_element(By.CLASS_NAME, 'artist').text

    # 가사 없는 경우 np.nan으로 처리
    lyric_element = driver.find_elements(By.CLASS_NAME, 'lyric')
    lyric =  lyric_element[0].text if lyric_element else np.nan

    when = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text
    like = driver.find_element(By.XPATH, '//*[@id="d_like_count"]').text
    genre = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]').text

    titles.append(title)
    artists.append(artist)
    lyrics.append(lyric)
    whens.append(when)
    likes.append(like)

    dj_rep = np.repeat(dj + '_' + str(i),len(titles))
  ## tag_rep = [tags for _ in range(len(titles))]

  df = pd.DataFrame({'플리':dj_rep, '제목':titles, '가수':artists, '가사':lyrics,
                      '장르':genre, '발매일':whens, '좋아요':likes})

  df['가사'] = df['가사'].str.replace('\n', ' ', regex=False)

  df.to_pickle(f'/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2024 Fall/추천시스템/Week 5 - 데이터 수집/DATA/Playlist/pickles/{dj}/{dj}_{i}.pkl')

  print(f"{i} 번째 : ", tags, playlist)

  # print(f'{dj}의 {i}번째 playlist 저장 완료했습니다 :)')

11 번째 :  ['#카페', '#집중', '#휴식', '#주말', '#작업', '#여유', '#매장음악', '#알앤비', '#일상', '#기분좋은'] 509181402
12 번째 :  ['#재즈', '#올드팝', '#커피', '#몽환', '#휴식', '#LP', '#카페', '#샹송', '#빈티지', '#힐링'] 530320779
13 번째 :  ['#카페', '#재즈', '#커피', '#브이로그', '#여유로운', '#잔잔한', '#편안한', '#휴식', '#빈티지', '#분위기'] 509592756
14 번째 :  ['#그루브', '#세련된', '#둠칫둠칫', '#편집숍', '#무드', '#라운지', '#느낌있는', '#감각적', '#분위기', '#혼술'] 520669363
15 번째 :  ['#프렌치팝', '#샹송', '#프랑스', '#낭만', '#로맨틱', '#유럽', '#여행', '#독특한', '#카페'] 511638076
16 번째 :  ['#밤', '#드라이브', '#새벽', '#느낌있는', '#힙한', '#분위기', '#외힙', '#베이스', '#칠아웃', '#야경'] 539368060
17 번째 :  ['#흥폭발', '#트렌디', '#신나는', '#댄스', '#자신감', '#운동', '#헬스', '#스타일리시', '#텐션업', '#런닝'] 522357934
18 번째 :  ['#힙한', '#매장', '#감각적인', '#분위기', '#편집샵', '#트렌디', '#라운지', '#클럽', '#헬스', '#느낌있는'] 527196526
19 번째 :  ['#드라이브', '#댄스', '#신나는', '#흥폭발하는', '#흥겨운', '#즐거운', '#트렌디', '#둠칫둠칫', '#레트로', '#추억'] 520189741
20 번째 :  ['#드라이브', '#신나는', '#휴가', '#시원한', '#청량', '#기분전환', '#여행', '#운전', '#즐거운', '#더위'] 509826550


In [4]:
df

NameError: name 'df' is not defined

In [5]:
import pandas as pd
import numpy as np
import pickle

In [7]:
df = pd.read_pickle('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2024 Fall/추천시스템/Week 5 - 데이터 수집/DATA/Playlist/pickles/boba/boba_10.pkl')
df.head()

,플리,제목,가수,가사,장르,발매일,좋아요
0,boba_10,Emotional Cowboy,Johnny Stimson,Woke up and my hotel was shaking I was laying ...,POP,2024.09.27,36
1,boba_10,Quarantined With You,Lawrence (로렌스),It’s a pretty scary time I’m sure we can agree...,POP,2021.03.30,271
2,boba_10,old timers (feat. Nieman),Sarah Kang,I wonder what will be on TV In the year 2063 W...,POP,2024.10.11,145
3,boba_10,Valentine,Laufey,I've rejected affection For years and years No...,POP,2022.08.29,556
4,boba_10,nuance,Luca minor,"Wish I said I love you late, I don’t know, it...",POP,2023.12.07,410


In [ ]:
boba_1
boba_2
boba_3
boba_4
boba_5
boba_6
boba_7
boba_8
boba_9
boba_10
boba_11
boba_12
boba_13
boba_14
boba_15
boba_16
boba_17
boba_18
boba_19
boba_20